### First start by installing necessary libraries like Roboflow and ultralytics

In [ ]:
# Install Roboflow for dataset storage and import
!pip install roboflow --upgrade

# Install nvidia-smi
!pip install nvidia-smi

# Install Ultralytics for yolo model access
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00


### Check GPU and RAM

In [ ]:
# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Check RAM
import psutil
ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sat Nov 15 02:45:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Import Roboflow and setup dataset workspace

In [ ]:
from roboflow import Roboflow

# Access Roboflow workspace and project via private api key
rf = Roboflow(api_key = 'eopByftdkdG3rvyTqNMU')
project = rf.workspace('teamdoggiedoo').project('petwastedataset-k4ogq')
version = project.version(1)

# Download dataset from roboflow in a yolov8 structured format (yolo11 uses same structure as v8)
dataset = version.download('yolov8')

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to PetWasteDataset-1 in yolov8:: 100%|██████████| 3202/3202 [00:00<00:00, 3716.74it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Perform Preprocessing on Dataset

In [ ]:
# NOTE: Supposedly, Ultralytics Yolo will apply augmentations on the fly during training

### Train yolo model on Training Dataset

In [ ]:
# Use Ultralytics YOLO CLI to train model
!yolo task=detect  \
mode=train \
model=yolo11n.pt \
data={dataset.location}/data.yaml \
epochs=100 # each epoch is a full pass through entire training dataset
imgsz=512 # imgsz controls resolution YOLo resizes images to during training

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PetWasteDataset-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0

### Validate Trained Model on Validation Dataset

In [ ]:
# Validate Model on validation portion of dataset using best performing weights
!yolo task=detect \
mode=val \
model=runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2741.4±916.6 MB/s, size: 120.3 KB)
val: Scanning /content/PetWasteDataset-1/valid/labels.cache... 323 images, 30 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 323/323 5.8Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 21/21 6.8it/s 3.1s
                   all        323        389      0.681      0.758       0.74      0.373
Speed: 1.1ms preprocess, 2.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


### Test Trained & Validated Model on Test Dataset

In [ ]:
# test model
!yolo task=detect \
mode=predict \
model=runs/detect/train/weights/best.pt \
conf=0.25 \
source={dataset.location}/test/images

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

image 1/147 /content/PetWasteDataset-1/test/images/034aa836-4bee-4368-b950-1a46a542568f_JPG.rf.6e4192c8c782a18ff90d51bb176cb91a.jpg: 640x480 1 dog-poop, 113.2ms
image 2/147 /content/PetWasteDataset-1/test/images/1000_F_211646955_f4brwMh2maQ0vmIjZ5KgFg11CVr5O5Mt_jpg.rf.09646bee04a63d579025843d1b732c49.jpg: 448x640 1 dog-poop, 76.6ms
image 3/147 /content/PetWasteDataset-1/test/images/19bd94ef-cb15-4b06-b739-4afe67d0e2ce_JPG.rf.89de87b581e06584d149f570ff25e13b.jpg: 640x480 2 dog-poops, 9.4ms
image 4/147 /content/PetWasteDataset-1/test/images/1_webp.rf.05b590524c9090ad567b6ede7d0cf582.jpg: 384x640 (no detections), 76.2ms
image 5/147 /content/PetWasteDataset-1/test/images/1d80dd79-b45b-4afc-9a3c-e2f611f4bdf3_JPG.rf.d99f043d2acbf679d6a2bf895558f9bc.jpg: 640x480 1 dog-poop, 9.4ms
image 6/147 /content/PetWasteDataset